In [42]:
import numpy as np
import sys 
sys.path.append(r'../../Python Script/')
from sympy import symbols, simplify, derive_by_array
from scipy.integrate import solve_ivp
from xLSINDy import *
from sympy.physics.mechanics import *
from sympy import *
import sympy
import torch
import HLsearch as HL
import matplotlib.pyplot as plt

In [43]:
states_dim = 4
states = ()
states_dot = ()
for i in range(states_dim):
    if(i<states_dim//2):
        states = states + (symbols('x{}'.format(i)),)
        states_dot = states_dot + (symbols('x{}_t'.format(i)),)
    else:
        states = states + (symbols('x{}_t'.format(i-states_dim//2)),)
        states_dot = states_dot + (symbols('x{}_tt'.format(i-states_dim//2)),)
print('states are:',states)
print('states derivatives are: ', states_dot)

#Turn from sympy to str
states_sym = states
states_dot_sym = states_dot
states = list(str(descr) for descr in states)
states_dot = list(str(descr) for descr in states_dot)

states are: (x0, x1, x0_t, x1_t)
states derivatives are:  (x0_t, x1_t, x0_tt, x1_tt)


In [44]:
#For friction force
x0 = Symbol(states[0], real=True)
x1 = Symbol(states[1], real=True)
x0_t = Symbol(states[2],real=True)
x1_t = Symbol(states[3],real=True)
q = sympy.Array([x0, x1])
qdot = sympy.Array([x0_t, x1_t])

#True Rayleigh Dissipation function
dummy = Symbol('a', real = True)
R = dummy #0.5*k1*x0_t**2 + 0.5*k2*(x1_t - x0_t)**2 #+ k1*Abs(x0_t) + k2*Abs(x1_t - x0_t)

#friction force
f_forcing = sympy.Matrix(derive_by_array(R, qdot)) 

In [45]:
#for lagrangian
x0 = dynamicsymbols(states[0], real=True)
x1 = dynamicsymbols(states[1], real=True)
x0_t = dynamicsymbols(states[0],1, real=True)
x1_t = dynamicsymbols(states[1],1, real=True)
T = symbols('T')
m = symbols('m')
L = symbols('L')
g = symbols('g')

#True Lagrangian
L = 0.5*m*L**2*(x0_t**2 + x1_t**2*sin(x0)**2) + m*g*L*cos(x0)
# Lagrange's method
LM = LagrangesMethod(L, [x0,x1])
LM.form_lagranges_equations()
i_forcing = LM.forcing #internal forcing and gravity
e_forcing = sympy.Matrix([0, T]) #external generalized force

In [46]:
# Substituting dynamic symbols

i_forcing = i_forcing.subs(x0_t, states_sym[2])
i_forcing = i_forcing.subs(x1_t, states_sym[3])
i_forcing = i_forcing.subs(x0, states_sym[0])
i_forcing = i_forcing.subs(x1, states_sym[1])

M = LM.mass_matrix
M = M.subs(x0, states_sym[0])
M = M.subs(x1, states_sym[1])

In [47]:
# Generating equation of motion
t_forcing = i_forcing + e_forcing - f_forcing
eom = M.inv()*sympy.Matrix(t_forcing)

In [48]:
eom = simplify(eom)

In [49]:
''' Please copy the string shown to the definition of equation in the function of double pendulum'''
for i in range(len(eom)):
    print('Equation ' + str(i) +': ' + str(eom[i]))
    print('\n')

Equation 0: 1.0*(L*x1_t**2*cos(x0) - g)*sin(x0)/L


Equation 1: 1.0*(-L**2*m*x0_t*x1_t*sin(2*x0) + T)/(L**2*m*sin(x0)**2)




In [50]:
import time

g = 9.81
m = 1
L,l = 1,1




def spherePend(t,y,T = 1.0):
    from numpy import sin, cos, sign
    x0,x1,x0_t,x1_t = y
    x0_tt = 1.0*(L*x1_t**2*cos(x0) - g)*sin(x0)/L
    x1_tt =  1.0*(-L**2*m*x0_t*x1_t*sin(2*x0) + T)/(L**2*m*sin(x0)**2)
    return x0_t,x1_t,x0_tt,x1_tt


def spherePendH(t,y,Moment=0.0):
    from numpy import sin, cos
    theta, psi, theta_t, psi_t = y
    theta_2t = sin(theta)*cos(theta)*psi_t**2 - (g/l)*sin(theta)
    psi_2t = -2*theta_t*psi_t*cos(theta)/sin(theta)
    return theta_t,psi_t,theta_2t,psi_2t

def generate_data(func, time, init_values):
    sol = solve_ivp(func,[time[0],time[-1]],init_values,t_eval=time, method='LSODA', rtol=1e-10,atol=1e-10)
    return sol.y.T, np.array([func(time[i],sol.y.T[i,:]) for i in range(sol.y.T.shape[0])],dtype=np.float64)

In [51]:
#Saving Directory
rootdir = "../../Spherical Pendulum/Data/Active/"

num_sample = 100
create_data = False
training = True
save = True
noiselevel = 6e-2

In [52]:
#Create training data
if(create_data):
    print("Creating Data . . .")
    X, Xdot = [], []
    Tau = []
    for i in range(num_sample):
        t = np.arange(0,5,0.01)
        theta = np.random.uniform(np.pi/3, np.pi/2)
        
        tau = np.zeros((len(t), 2))
        tau[:,1] = 1.0    
        y0=np.array([theta, 0, 0, np.pi])
        x,xdot = generate_data(spherePend,t,y0)
        
        #Omega.append(omega)
        Tau.append(tau)
        X.append(x)
        Xdot.append(xdot)

    X = np.vstack(X)
    Xdot = np.vstack(Xdot)
    Tau = np.vstack(Tau)
    if(save==True):
        np.save(rootdir + "X.npy", X)
        np.save(rootdir + "Xdot.npy",Xdot)
        np.save(rootdir + "Tau.npy", Tau)
else:
    X = np.load(rootdir + "X.npy")
    Xdot = np.load(rootdir + "Xdot.npy")
    Tau = np.load(rootdir + "Tau.npy")

In [53]:
#adding noise
mu, sigma = 0, noiselevel
noise = np.random.normal(mu, sigma, X.shape[0])
for i in range(X.shape[1]):
    X[:,i] = X[:,i]+noise
    Xdot[:,i] = Xdot[:,i]+noise

In [54]:
states_dim = 4
states = ()
states_dot = ()
for i in range(states_dim):
    if(i<states_dim//2):
        states = states + (symbols('x{}'.format(i)),)
        states_dot = states_dot + (symbols('x{}_t'.format(i)),)
    else:
        states = states + (symbols('x{}_t'.format(i-states_dim//2)),)
        states_dot = states_dot + (symbols('x{}_tt'.format(i-states_dim//2)),)
print('states are:',states)
print('states derivatives are: ', states_dot)

states are: (x0, x1, x0_t, x1_t)
states derivatives are:  (x0_t, x1_t, x0_tt, x1_tt)


In [55]:
#Turn from sympy to str
states_sym = states
states_dot_sym = states_dot
states = list(str(descr) for descr in states)
states_dot = list(str(descr) for descr in states_dot)

In [56]:
#build function expression for the library in str
exprdummy = HL.buildFunctionExpressions(1,states_dim,states,use_sine=True)
polynom = exprdummy[1:4]
trig = [exprdummy[4], exprdummy[6]]
polynom = HL.buildFunctionExpressions(2,len(polynom),polynom)
trig = HL.buildFunctionExpressions(2, len(trig),trig)
product = []
for p in polynom:
    for t in trig:
        product.append(p + '*' + t)
expr = polynom + trig + product

In [57]:
### Boundaries for debugging with only the correct terms ###


In [58]:
#Creating library tensor
Zeta, Eta, Delta = LagrangianLibraryTensor(X,Xdot,expr,states,states_dot, scaling=False)

In [59]:
## Case I, input torque provided##
expr = np.array(expr)
i0 = np.where(expr == 'x0_t**2*cos(x0)**2')[0]

idx = np.arange(0,len(expr))
idx = np.delete(idx,[0])
expr = np.delete(expr,[i0])


#non-penalty index from prev knowledge
i1 = np.where(expr == 'x0_t**2')[0][0]
i2 = np.where(expr == 'cos(x0)')[0][0]

nonpenaltyidx = [i1,i2]

expr = expr.tolist()

Zeta = Zeta[:,:,idx,:]
Eta = Eta[:,:,idx,:]
Delta = Delta[:,idx,:]

In [60]:
#Moving to Cuda
device = 'cuda:0'

Zeta = Zeta.to(device)
Eta = Eta.to(device)
Delta = Delta.to(device)

#computing upsilon
UpsilonR = Upsilonforward(Zeta, Eta, Delta, Xdot, device)

In [61]:
xi_L = torch.ones(len(expr), device=device).data.uniform_(-10,10)
prevxi_L = xi_L.clone().detach()

In [62]:
def loss(pred, targ):
    loss = torch.mean((pred - targ)**2) 
    return loss 

In [63]:
def clip(w, alpha):
    clipped = torch.minimum(w,alpha)
    clipped = torch.maximum(clipped,-alpha)
    return clipped

def proxL1norm(w_hat, alpha, nonpenaltyidx):
    if(torch.is_tensor(alpha)==False):
        alpha = torch.tensor(alpha)
    w = w_hat - clip(w_hat,alpha)
    for idx in nonpenaltyidx:
        w[idx] = w_hat[idx]
    return w

In [64]:
def training_loop(coef, prevcoef, UpsilonR, Tau, xdot, bs, lr, lam, momentum=True):
    loss_list = []
    tl = xdot.shape[0]
    n = xdot.shape[1]

    if(torch.is_tensor(xdot)==False):
        xdot = torch.from_numpy(xdot).to(device).float()
    if(torch.is_tensor(Tau)==False):
        Tau = torch.from_numpy(Tau).to(device).float()

    v = coef.clone().detach().requires_grad_(True)
    prev = v
    
    for i in range(tl//bs):
                
        #computing acceleration with momentum
        if(momentum==True):
            vhat = (v + ((i-1)/(i+2))*(v - prev)).clone().detach().requires_grad_(True)
        else:
            vhat = v.requires_grad_(True).clone().detach().requires_grad_(True)
   
        prev = v

        #Computing loss
        upsilonR = UpsilonR[:,:,i*bs:(i+1)*bs]
        tau = Tau[i*bs:(i+1)*bs]


        #forward
        pred = torch.einsum('jkl,k->jl', upsilonR, vhat)
        targ = tau.T
        
        lossval = loss(pred, targ)
        
        #Backpropagation
        lossval.backward()

        with torch.no_grad():
            v = vhat - lr*vhat.grad
            v = (proxL1norm(v,lr*lam,nonpenaltyidx))
            
            # Manually zero the gradients after updating weights
            vhat.grad = None
        
        
    
        
        loss_list.append(lossval.item())
    print("Average loss : " , torch.tensor(loss_list).mean().item())
    return v, prevcoef, torch.tensor(loss_list).mean().item()

In [82]:
Epoch = 500
i = 0
lr = 1e-4
lam = 0
temp = 200
while(i<=Epoch):
    print("Epoch "+str(i) + "/" + str(Epoch))
    print("Learning rate : ", lr)
    xi_L, prevxi_L, lossitem= training_loop(xi_L,prevxi_L,UpsilonR,Tau,Xdot,128,lr=lr,lam=lam)
    temp = lossitem
    i+=1

Epoch 0/500
Learning rate :  0.0001
Average loss :  0.1515112668275833
Epoch 1/500
Learning rate :  0.0001
Average loss :  0.15149714052677155
Epoch 2/500
Learning rate :  0.0001
Average loss :  0.151499941945076
Epoch 3/500
Learning rate :  0.0001
Average loss :  0.15149955451488495
Epoch 4/500
Learning rate :  0.0001
Average loss :  0.15149687230587006
Epoch 5/500
Learning rate :  0.0001
Average loss :  0.1515008807182312
Epoch 6/500
Learning rate :  0.0001
Average loss :  0.1514991819858551
Epoch 7/500
Learning rate :  0.0001
Average loss :  0.15149876475334167
Epoch 8/500
Learning rate :  0.0001
Average loss :  0.15149830281734467
Epoch 9/500
Learning rate :  0.0001
Average loss :  0.15149793028831482
Epoch 10/500
Learning rate :  0.0001
Average loss :  0.15149547159671783
Epoch 11/500
Learning rate :  0.0001
Average loss :  0.15149778127670288
Epoch 12/500
Learning rate :  0.0001
Average loss :  0.15150022506713867
Epoch 13/500
Learning rate :  0.0001
Average loss :  0.15150021016

In [83]:
## Thresholding
threshold = 1e-2
surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
expr = np.array(expr)[surv_index].tolist()

xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
prevxi_L = xi_L.clone().detach()

## obtaining analytical model
xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=2)
L = HL.generateExpression(xi_Lcpu,expr,threshold=1e-3)
print("Result stage 1: ", simplify(L))

Result stage 1:  0.03*x0_t**2 - 0.03*x1**2 + 0.03*x1_t**2*sin(x0)**2 + 0.51*cos(x0)


In [84]:
Zeta, Eta, Delta = LagrangianLibraryTensor(X,Xdot,expr,states,states_dot, scaling=False)

expr = np.array(expr)
i1 = np.where(expr == 'x0_t**2')[0][0]
i2 = np.where(expr == 'cos(x0)')[0][0]

nonpenaltyidx = [i1,i2]


Zeta = Zeta.to(device)
Eta = Eta.to(device)
Delta = Delta.to(device)

#computing upsilon
UpsilonR = Upsilonforward(Zeta, Eta, Delta, Xdot, device)

In [ ]:
### Debugging computation
expr = np.array(expr)
i0 = np.where(expr == 'x0_t**2')[0][0]
i1 = np.where(expr == 'cos(x0)')[0][0]
i2 = np.where(expr == 'x1_t**2*sin(x0)**2')[0][0]

expr = [expr[i0],expr[i1],expr[i2]]


In [ ]:
#Creating library tensor
Zeta, Eta, Delta = LagrangianLibraryTensor(X,Xdot,expr,states,states_dot, scaling=False)

In [ ]:
#define the true coefficient
xi_True = torch.ones(len(expr))
xi_True[0] = 0.5
xi_True[1] = 9.81
xi_True[2] = 0.5

In [ ]:
#Moving to Cuda
device = 'cuda:0'

Zeta = Zeta.to(device)
Eta = Eta.to(device)
Delta = Delta.to(device)

xi_True = xi_True.to(device)

In [ ]:
#compute tau prediction
xdot = torch.from_numpy(Xdot).to(device).float()
UpsilonL = Upsilonforward(Zeta, Eta, Delta, xdot, device)
TauPred = torch.einsum('jkl,k->jl', UpsilonL, xi_L).detach().cpu().numpy().T


TauEL = ELforward(xi_L, Zeta, Eta, Delta, xdot, device).detach().cpu().numpy().T

In [ ]:
#plot the figures
left_boundary = 1000
right_boundary = 2000
t = np.arange(left_boundary,right_boundary)
plt.plot(t,Tau[left_boundary:right_boundary,0])
plt.plot(t,TauPred[left_boundary:right_boundary,0])
plt.plot(t,TauEL[left_boundary:right_boundary,0])
plt.show()

plt.plot(t,Tau[left_boundary:right_boundary,1])
plt.plot(t,TauPred[left_boundary:right_boundary,1])
plt.plot(t,TauEL[left_boundary:right_boundary,1])
plt.show()

In [ ]:
plt.plot(t, Xdot[left_boundary:right_boundary,2])
plt.show()

In [ ]:
t = np.arange(0,5,0.01)
theta = np.random.uniform(np.pi/3, np.pi/2)

tau = np.zeros((len(t), 2))    
y0=np.array([theta, 0, 0, np.pi])
x,xdot = generate_data(spherePend,t,y0)
x_, xdot_ = generate_data(spherePendH,t,y0)

In [ ]:
plt.plot(t,x[:,0])
plt.plot(t,x_[:,0])
plt.show()

plt.plot(t,x[:,1])
plt.plot(t,x_[:,1])
plt.show()

plt.plot(t,x[:,2])
plt.plot(t,x_[:,2])
plt.show()

plt.plot(t,x[:,3])
plt.plot(t,x_[:,3])
plt.show()

plt.plot(t,xdot[:,2])
plt.plot(t,xdot_[:,2])
plt.show()

plt.plot(t,xdot[:,3])
plt.plot(t,xdot_[:,3])
plt.show()

In [ ]:
### Debugging for training with only known terms
expr = np.array(expr)
i0 = np.where(expr == 'x0_t**2')[0][0]
i1 = np.where(expr == 'cos(x0)')[0][0]
i2 = np.where(expr == 'x1_t**2*sin(x0)**2')[0][0]

expr = [expr[i0],expr[i1],expr[i2]]

#non-penalty index from prev knowledge
expr = np.array(expr)
i4 = np.where(expr == 'x0_t**2')[0][0]
i5 = np.where(expr == 'cos(x0)')[0][0]
nonpenaltyidx = [i4,i5]

In [ ]:
#Creating library tensor
Zeta, Eta, Delta = LagrangianLibraryTensor(X,Xdot,expr,states,states_dot, scaling=False)

In [ ]:
#Moving to Cuda
device = 'cuda:0'

Zeta = Zeta.to(device)
Eta = Eta.to(device)
Delta = Delta.to(device)

In [ ]:
#initialize coefficient
xi_L = torch.ones(len(expr), device=device).data.uniform_(-10,10)
prevxi_L = xi_L.clone().detach()

In [ ]:
#compute Upsilon
xdot = torch.from_numpy(Xdot).to(device).float()
UpsilonR = Upsilonforward(Zeta, Eta, Delta, Xdot, device)